In [ ]:
import pandas as pd
import os

In [ ]:
# Function to read stock data from an Excel file and skip the first 100 days
def get_stock_data_from_excel(file_path):
    all_stock_data = pd.read_excel(file_path, sheet_name=None)
    stock_data = {}

    # Get all sheet names
    all_sheets = list(all_stock_data.keys())

    # Get sheet names starting from the 150th sheet (if needed, uncomment)
    # sheets_to_read = all_sheets[150:]
    sheets_to_read = all_sheets

    for sheet_name in sheets_to_read:
        data = all_stock_data[sheet_name]
        data['Date'] = pd.to_datetime(data['Date'])
        data.set_index('Date', inplace=True)

        # Skip the first 100 rows (assuming one row per day)
        data = data.iloc[100:]

        stock_data[sheet_name] = data

    return stock_data


In [ ]:
# Function to read EMA combinations from an Excel file
def get_ema_combinations(file_path):
    return pd.read_excel(file_path)

In [ ]:
# Function to calculate EMA for a given stock's dataframe
def calculate_ema(data, period):
    return data['Close'].ewm(span=period, adjust=False).mean()

In [ ]:
# Function to calculate MACD and signal line
def calculate_macd(data, fast_ema_period, slow_ema_period, signal_period):
    data['EMA_' + str(fast_ema_period)] = calculate_ema(data, fast_ema_period)
    data['EMA_' + str(slow_ema_period)] = calculate_ema(data, slow_ema_period)
    data['MACD'] = data['EMA_' + str(fast_ema_period)] - data['EMA_' + str(slow_ema_period)]
    data['Signal Line'] = data['MACD'].ewm(span=signal_period, adjust=False).mean()

    return data

In [ ]:
# Function to create SCRIP DataFrame based on MACD crossover signals
def create_scrip_dataframe(data, fast_ema_period, slow_ema_period, signal_period):
    data = calculate_macd(data, fast_ema_period, slow_ema_period, signal_period)

    # Generate 'Long' or 'Short' signals based on MACD and Signal Line crossovers
    data['Long / Short'] = data.apply(lambda row: 'Long' if row['MACD'] > row['Signal Line'] else 'Short', axis=1)

    # Initialize Signal No. and increment on position change
    data['Signal No.'] = 0
    signal_no = 0
    previous_position = None
    for i in range(len(data)):
        current_position = data.iloc[i]['Long / Short']
        if current_position != previous_position:
            signal_no += 1
        data.at[data.index[i], 'Signal No.'] = signal_no
        previous_position = current_position

    return data

In [ ]:
# Function to create the Trade_Results DataFrame collectively for all trades (Entry at the Open)
def create_trade_results(scrip_df):
    trades=[]
    grouped = scrip_df.groupby('Signal No.')

    for signal_no, group in grouped:
        #Don't process signals below signal number 3 and ignore them
        if signal_no < 3:
            continue

        #Exclude the last signal for all trades combined
        if signal_no == scrip_df['Signal No.'].max():
            break

        entry = group.iloc[0]
        next_signal_entry = grouped.get_group(signal_no + 1).iloc[0]

        #Get the original signal
        original_signal = entry['Long / Short']

        #Entry at the open
        entry_index = scrip_df.index.get_loc(entry.name) + 1
        exit_index = scrip_df.index.get_loc(next_signal_entry.name) + 1

        # Ensure the index is within bounds
        if entry_index >= len(scrip_df) or exit_index >= len(scrip_df):
            continue

        entry_row = scrip_df.iloc[entry_index]
        exit_row = scrip_df.iloc[exit_index]

        #New entry and exit based on entry at the open
        entry_date = entry_row.name
        exit_date = exit_row.name

        entry_price = entry_row['Open']
        exit_price = exit_row['Open']

        #Calculate the duration of the trade
        days_in_trade = (exit_date - entry_date).days

        # Determine if the signal is a whipsaw signal
        whipsaw_signal_count = scrip_df[scrip_df['Signal No.'] == signal_no].shape[0]
        whipsaw_signal = 'YES' if whipsaw_signal_count == 1 else 'NO'

        # Determine the range of the trade while excluding the entry date
        period_data = scrip_df[(scrip_df.index >= entry_date) & (scrip_df.index < exit_date)]

        # Calculate highest or lowest point based on signal type
        highest_lowest_point = period_data['High'].max() if original_signal == 'Long' else period_data['Low'].min()
        highest_point_percent = ((highest_lowest_point - entry_price) / entry_price * 100) if original_signal == 'Long' else ((entry_price - highest_lowest_point) / entry_price * 100)

        # Determine if the highest point occured on the opening day
        opening_highest = 'YES' if highest_lowest_point <= 0.0 else 'NO'

        trades.append({
            'Signal No.': signal_no,
            'Type of Signal': original_signal,
            'Entry Date': entry_date,
            'Exit Date': exit_date,
            'Entry Price': entry_price,
            'Exit Price': exit_price,
            'Days in Trade': days_in_trade,
            'Pure Signal P&L': 0.0,  # Placeholder, will be calculated separately
            'Pure Signal Won/Lost': '',  # Placeholder, will be calculated separately
            'Highest Point/Lowest Point': highest_lowest_point,
            'Highest Point %': highest_point_percent,
            'Opening = Highest': opening_highest,
            'Whipsaw Signal': whipsaw_signal,
            'Opening Equity': 0.0,  # Placeholder, will be calculated separately
            'Shares Bought': 0,  # Placeholder, will be calculated separately
            'Total Price Paid': 0.0,  # Placeholder, will be calculated separately
            'Total Price Got': 0.0,  # Placeholder, will be calculated separately
            'Transaction Cost': 0.0,  # Placeholder, will be calculated separately
            'Closing Equity': 0.0,  # Placeholder, will be calculated separately
        })

    trade_results_df = pd.DataFrame(trades)
    return trade_results_df

In [ ]:
# Function to calculate financial metrics for a given trade results DataFrame
def calculate_financial_metrics(trade_results_df, starting_equity):
    previous_equity = starting_equity
    for i, row in trade_results_df.iterrows():
        shares_bought = round(previous_equity / row['Entry Price'])
        total_price_paid = shares_bought * row['Entry Price']
        total_price_got = shares_bought * row['Exit Price']
        transaction_cost = total_price_paid * 0.002
        pnl = ((row['Exit Price'] - row['Entry Price']) * shares_bought) - transaction_cost if row['Type of Signal'] == 'Long' else ((row['Entry Price'] - row['Exit Price']) * shares_bought) - transaction_cost
        won_lost = 'Won' if pnl > 0 else 'Lost'
        closing_equity = previous_equity + pnl

        # Update the dataframe
        trade_results_df.at[i, 'Opening Equity'] = previous_equity
        trade_results_df.at[i, 'Shares Bought'] = shares_bought
        trade_results_df.at[i, 'Total Price Paid'] = total_price_paid
        trade_results_df.at[i, 'Total Price Got'] = total_price_got
        trade_results_df.at[i, 'Transaction Cost'] = transaction_cost
        trade_results_df.at[i, 'Pure Signal P&L'] = pnl
        trade_results_df.at[i, 'Pure Signal Won/Lost'] = won_lost
        trade_results_df.at[i, 'Closing Equity'] = closing_equity

        previous_equity = closing_equity

    return trade_results_df

In [ ]:
# Function to create the Analysis DataFrame for a given set of trades
def create_analysis_dataframe(trade_results_df, signal_type, stock_symbol, fast_ema, slow_ema, signal_period, starting_equity):
    total_trades = len(trade_results_df)
    winning_trades = trade_results_df[trade_results_df['Pure Signal Won/Lost'] == 'Won']
    losing_trades = trade_results_df[trade_results_df['Pure Signal Won/Lost'] == 'Lost']
    winning_percentage = (len(winning_trades) / total_trades) * 100 if total_trades > 0 else 0
    losing_percentage = (len(losing_trades) / total_trades) * 100 if total_trades > 0 else 0
    num_whipsaws = trade_results_df[trade_results_df['Whipsaw Signal'] == 'Yes'].shape[0]
    num_opening_highest = trade_results_df[trade_results_df['Opening = Highest'] == 'Yes'].shape[0]
    twentieth_percentile = trade_results_df['Highest Point %'].quantile(0.2) if total_trades > 0 else 0

    # Calculate CAGR (Compound Annual Growth Rate)
    if total_trades > 0:
        num_years = trade_results_df['Days in Trade'].sum() / 365
        ending_equity = trade_results_df.iloc[-1]['Closing Equity']
        cagr = (((ending_equity / starting_equity) ** (1 / num_years)) - 1) * 100 if num_years > 0 else 0
    else:
        cagr = 0

    analysis_data = {
        'Scrip Name': stock_symbol,
        'Fast EMA': fast_ema,
        'Slow EMA': slow_ema,
        'Signal Period': signal_period,
        '# Signals': total_trades,
        'Signal Type': signal_type,
        'CAGR- Return': cagr,
        'Winning %': winning_percentage,
        'Losing %': losing_percentage,
        '# Whipsaws': num_whipsaws,
        '# Opening = Highest': num_opening_highest,
        '20th Percentile': twentieth_percentile
    }
    analysis_df = pd.DataFrame([analysis_data])
    return analysis_df, twentieth_percentile

In [ ]:
def new_trade_results(scrip_df, twentieth_percentile_long, twentieth_percentile_short):
    new_trades = []

    grouped = scrip_df.groupby('Signal No.')

    for signal_no, group in grouped:
        #Don't process signals below signal number 3 and ignore them
        if signal_no < 3:
            continue

        #Exclude the last signal for all trades combined
        if signal_no == scrip_df['Signal No.'].max():
            break

        entry = group.iloc[0]
        next_signal_entry = grouped.get_group(signal_no + 1).iloc[0]

        #Get the original signal
        original_signal = entry['Long / Short']

        #Entry at the open
        entry_index = scrip_df.index.get_loc(entry.name) + 1
        exit_index = scrip_df.index.get_loc(next_signal_entry.name) + 1

        # Ensure the index is within bounds
        if entry_index >= len(scrip_df) or exit_index >= len(scrip_df):
            continue

        entry_row = scrip_df.iloc[entry_index]
        exit_row = scrip_df.iloc[exit_index]

        #New entry and exit based on entry at the open
        entry_date = entry_row.name
        exit_date = exit_row.name

        entry_price = entry_row['Open']
        exit_price = exit_row['Open']

        #Calculate the duration of the trade
        days_in_trade = (exit_date - entry_date).days

        # Determine range of trade
        period_data = scrip_df[(scrip_df.index >= entry_date) & (scrip_df.index < exit_date)]

        # Initialize the variables
        highest_point_percent = 0.0
        lowest_point_percent = 0.0
        achieved_20th_percentile = ''
        opening_highest = ''
        opening_lowest = ''

        # Calculate highest point percentage based on signal type
        if original_signal == 'Long':
            highest_point = period_data['High'].max()
            highest_point_percent = ((highest_point - entry_price) / entry_price * 100)
            achieved_20th_percentile = 'YES' if highest_point_percent >= twentieth_percentile_long else 'NO'
            opening_highest = 'YES' if highest_point_percent <= 0.0 else 'NO'
        else:
            lowest_point = period_data['Low'].min()
            lowest_point_percent = ((entry_price - lowest_point) / entry_price * 100)
            achieved_20th_percentile = 'YES' if lowest_point_percent >= twentieth_percentile_short else 'NO'
            opening_lowest = 'YES' if lowest_point_percent <= 0.0 else 'NO'

        # Initialize more variables

        date_of_highest_point = ''

        days_to_reach_highest_point = ''
        days_to_reach_lowest_point = ''

        date_of_lowest_point = ''

        # Find the date of highest and lowest point
        if achieved_20th_percentile == 'YES':
            if original_signal == 'Long':
                date_of_highest_point = period_data[period_data['High'] == highest_point].index[0]
                days_to_reach_highest_point = (date_of_highest_point - entry_date).days
            else:
                date_of_lowest_point = period_data[period_data['Low'] == lowest_point].index[0]
                days_to_reach_lowest_point = (date_of_lowest_point - entry_date).days

        new_trades.append({
            'Signal No.': signal_no,
            'Type of Signal': original_signal,
            'Entry Date': entry_date,
            'Exit Date': exit_date,
            'Entry Price': entry_price,
            'Exit Price': exit_price,
            'Days in Trade': days_in_trade,
            'Highest Point' if original_signal == 'Long' else 'Lowest Point': highest_point if original_signal == 'Long' else lowest_point,
            'Highest Point %' if original_signal == 'Long' else 'Lowest Point %': highest_point_percent if original_signal == 'Long' else lowest_point_percent,
            'Date of reaching Highest Point' if original_signal == 'Long' else 'Date of reaching Lowest Point': date_of_highest_point if original_signal == 'Long' else date_of_lowest_point,
            'Days for reaching Highest Point' if original_signal == 'Long' else 'Days for reaching Lowest Point': days_to_reach_highest_point if original_signal == 'Long' else days_to_reach_lowest_point,
            '20th Percentile Reached?': achieved_20th_percentile,
            'Opening = Highest' if original_signal == 'Long' else 'Opening = Lowest': opening_highest if original_signal == 'Long' else opening_lowest
        })

    #Convert to dataframe
    new_trades_df = pd.DataFrame(new_trades)
    return new_trades_df

In [ ]:
def success_sheet(scrip_df, new_long_20th_achieved_df, new_short_20th_achieved_df, twentieth_percentile_long, twentieth_percentile_short):
    # Extract relevant columns from long and short trades dataframes
    long_success = new_long_20th_achieved_df[['Signal No.', 'Type of Signal', 'Entry Date', 'Exit Date', 'Entry Price', 'Highest Point', 'Highest Point %', 'Date of reaching Highest Point', 'Days for reaching Highest Point', '20th Percentile Reached?', 'Opening = Highest']].copy()
    short_success = new_short_20th_achieved_df[['Signal No.', 'Type of Signal', 'Entry Date', 'Exit Date', 'Entry Price', 'Lowest Point', 'Lowest Point %', 'Date of reaching Lowest Point', 'Days for reaching Lowest Point', '20th Percentile Reached?', 'Opening = Lowest']].copy()

    # Add Column for Target for 20th Percentile
    long_success.loc[:, 'Target for 20th Percentile'] = long_success['Entry Price'] * (1 + twentieth_percentile_long / 100)
    short_success.loc[:, 'Target for 20th Percentile'] = short_success['Entry Price'] * (1 - twentieth_percentile_short / 100)   # NEW CHANGE

    # Add column for Target Reached On
    long_success.loc[:, 'Target Reached On'] = long_success.apply(
        lambda row: scrip_df[
            (scrip_df['Signal No.'] == row['Signal No.']) &
            (scrip_df['High'] >= row['Target for 20th Percentile']) &
            (scrip_df.index >= row['Entry Date'])
        ].index.min() if not scrip_df[
            (scrip_df['Signal No.'] == row['Signal No.']) &
            (scrip_df['High'] >= row['Target for 20th Percentile']) &
            (scrip_df.index >= row['Entry Date'])
        ].empty else None,
        axis=1
    )

    short_success.loc[:, 'Target Reached On'] = short_success.apply(
        lambda row: scrip_df[
            (scrip_df['Signal No.'] == row['Signal No.']) &
            (scrip_df['Low'] <= row['Target for 20th Percentile']) &
            (scrip_df.index >= row['Entry Date'])
        ].index.min() if not scrip_df[
            (scrip_df['Signal No.'] == row['Signal No.']) &
            (scrip_df['Low'] <= row['Target for 20th Percentile']) &
            (scrip_df.index >= row['Entry Date'])
        ].empty else None,
        axis=1
    )

    # Add column for Days for 20th Percentile
    long_success.loc[:, 'Days for 20th Percentile'] = long_success.apply(lambda row: (row['Target Reached On'] - row['Entry Date']).days if pd.notnull(row['Target Reached On']) else None, axis=1)
    short_success.loc[:, 'Days for 20th Percentile'] = short_success.apply(lambda row: (row['Target Reached On'] - row['Entry Date']).days if pd.notnull(row['Target Reached On']) else None, axis=1)

    # Add column for Lowest point on the way to the 20th Percentile
    long_success.loc[:, 'Lowest point on the way to the 20th Percentile'] = long_success.apply(lambda row: scrip_df[(scrip_df['Signal No.'] == row['Signal No.']) & (scrip_df.index >= row['Entry Date']) & (scrip_df.index <= row['Target Reached On'])]['Low'].min() if pd.notnull(row['Target Reached On']) else None, axis=1)
    short_success.loc[:, 'Highest point on the way to the 20th Percentile'] = short_success.apply(lambda row: scrip_df[(scrip_df['Signal No.'] == row['Signal No.']) & (scrip_df.index >= row['Entry Date']) & (scrip_df.index <= row['Target Reached On'])]['High'].max() if pd.notnull(row['Target Reached On']) else None, axis=1)

    # Add column for Lowest Point %
    long_success.loc[:, 'Lowest Point %'] = long_success.apply(lambda row: (row['Lowest point on the way to the 20th Percentile'] - row['Entry Price']) / row['Entry Price'] * 100 if pd.notnull(row['Lowest point on the way to the 20th Percentile']) else None, axis=1)
    short_success.loc[:, 'Highest Point %'] = short_success.apply(lambda row: (row['Entry Price'] - row['Highest point on the way to the 20th Percentile']) / row['Entry Price'] * 100 if pd.notnull(row['Highest point on the way to the 20th Percentile']) else None, axis=1)

    return long_success, short_success

In [ ]:
def failure_sheet(scrip_df, new_long_20th_not_achieved_df, new_short_20th_not_achieved_df, twentieth_percentile_long, twentieth_percentile_short):
    # Extract relevant columns from long and short trades dataframes
    long_failure = new_long_20th_not_achieved_df[['Signal No.', 'Type of Signal', 'Entry Date', 'Exit Date', 'Entry Price', 'Exit Price', 'Days in Trade', 'Highest Point', 'Highest Point %', '20th Percentile Reached?', 'Opening = Highest']].copy()
    short_failure = new_short_20th_not_achieved_df[['Signal No.', 'Type of Signal', 'Entry Date', 'Exit Date', 'Entry Price', 'Exit Price', 'Days in Trade', 'Lowest Point', 'Lowest Point %',  '20th Percentile Reached?', 'Opening = Lowest']].copy()

    # Calculate the lowest point in the trade for long and short failures
    long_failure.loc[:, 'Lowest Point in the Trade'] = long_failure.apply(lambda row: scrip_df[(scrip_df.index >= row['Entry Date']) & (scrip_df.index <= row['Exit Date'])]['Low'].min(), axis=1)
    short_failure.loc[:, 'Lowest Point in the Trade'] = short_failure.apply(lambda row: scrip_df[(scrip_df.index >= row['Entry Date']) & (scrip_df.index <= row['Exit Date'])]['Low'].min(), axis=1)

    # Calculate lowest point % for long and short failures
    long_failure.loc[:, 'Lowest Point % (20th Not Achieved)'] = long_failure.apply(lambda row: (row['Lowest Point in the Trade'] - row['Entry Price']) / row['Entry Price'] * 100, axis=1)
    short_failure.loc[:, 'Lowest Point % (20th Not Achieved)'] = short_failure.apply(lambda row: (row['Lowest Point in the Trade'] - row['Entry Price']) / row['Entry Price'] * 100, axis=1)

    return long_failure, short_failure

In [ ]:
def final_output_long(new_long_trade_results_df, long_success, long_failure):
    # 80th Percentile time of highest point
    new_long_trade_results_df['Days for reaching Highest Point'] = pd.to_numeric(new_long_trade_results_df['Days for reaching Highest Point'], errors='coerce')
    time_highest_point_80th = new_long_trade_results_df['Days for reaching Highest Point'].quantile(0.8)

    # 80th Percentile time of 20th Percentile
    long_success['Days for 20th Percentile'] = pd.to_numeric(long_success['Days for 20th Percentile'], errors='coerce')
    time_20th_percentile_80th = long_success['Days for 20th Percentile'].quantile(0.8)

    # 20th Percentile of lowest point for achievement of 20th Percentile
    lowest_point_20th_achieved_20th = long_success['Lowest Point %'].quantile(0.2)

    # 20th Percentile of lowest point for trades not achieving the 20th Percentile
    lowest_point_20th_not_achieved_20th = long_failure['Lowest Point % (20th Not Achieved)'].quantile(0.2)

    return time_highest_point_80th, time_20th_percentile_80th, lowest_point_20th_achieved_20th, lowest_point_20th_not_achieved_20th

In [ ]:
def final_output_short(new_short_trade_results_df, short_success, short_failure):
    # 80th Percentile time of lowest point
    new_short_trade_results_df['Days for reaching Lowest Point'] = pd.to_numeric(new_short_trade_results_df['Days for reaching Lowest Point'], errors='coerce')
    time_lowest_point_80th = new_short_trade_results_df['Days for reaching Lowest Point'].quantile(0.8)

    # 80th Percentile time of 20th Percentile
    short_success['Days for 20th Percentile'] = pd.to_numeric(short_success['Days for 20th Percentile'], errors='coerce')
    time_20th_percentile_80th = short_success['Days for 20th Percentile'].quantile(0.8)

    # 20th Percentile of highest point for achievement of 20th Percentile
    highest_point_20th_achieved_20th = short_success['Highest Point %'].quantile(0.2)

    # 20th Percentile of lowest point for trades not achieving the 20th Percentile
    lowest_point_20th_not_achieved_20th = short_failure['Lowest Point % (20th Not Achieved)'].quantile(0.2)

    return time_lowest_point_80th, time_20th_percentile_80th, highest_point_20th_achieved_20th, lowest_point_20th_not_achieved_20th


In [ ]:
# Path to the Excel file containing stock data
file_path = '/content/drive/MyDrive/Data/DATA_FOR_TESTING_P2.xlsx'

In [ ]:
# Path to the Excel file containing EMA combinations
ema_combinations_file_path ='/content/drive/MyDrive/Data/EMA_COMBOS.xlsx'

In [ ]:
# Get stock data from Excel file
stock_data = get_stock_data_from_excel(file_path)

In [ ]:
# Get EMA combinations from Excel file
ema_combinations = get_ema_combinations(ema_combinations_file_path)

In [ ]:
starting_equity = 100000  # Initial equity

In [ ]:
# Initialize lists to store all Long and Short results for all stocks
all_long_final_output = []
all_short_final_output = []

In [ ]:
count=1

In [ ]:
# Run the analysis for each selected stock
for stock_symbol in stock_data.keys():
    print(f"{count}. Processing {stock_symbol}...")
    count+=1

    try:
        data = stock_data[stock_symbol]

        # Iterate over each combination of Fast and Slow EMAs
        for _, ema_combo in ema_combinations.iterrows():
            fast_ema_period = int(ema_combo['Fast EMA'])
            slow_ema_period = int(ema_combo['Slow EMA'])

            # Iterate over signal periods from 5 to 55 (in range(5,31))
            for signal_period in [5,8,13,21,25,29,34]:
                # Run the SCRIP dataframe creation
                scrip_df = create_scrip_dataframe(data, fast_ema_period, slow_ema_period, signal_period)

                # Generate trade results
                trade_results_df = create_trade_results(scrip_df)

                # Separate the trade results into Long and Short trade results
                long_trade_results_df = trade_results_df[trade_results_df['Type of Signal'] == 'Long'].reset_index(drop=True)
                short_trade_results_df = trade_results_df[trade_results_df['Type of Signal'] == 'Short'].reset_index(drop=True)

                # Calculate financial metrics
                long_trade_results_df = calculate_financial_metrics(long_trade_results_df, starting_equity)
                short_trade_results_df = calculate_financial_metrics(short_trade_results_df, starting_equity)

                # Generate analysis for Long and Short trade results separately
                long_analysis_df, twentieth_percentile_long = create_analysis_dataframe(long_trade_results_df, 'Long', stock_symbol, fast_ema_period, slow_ema_period, signal_period, starting_equity)
                short_analysis_df, twentieth_percentile_short = create_analysis_dataframe(short_trade_results_df, 'Short', stock_symbol, fast_ema_period, slow_ema_period, signal_period, starting_equity)


                #GENERATE NEW TRADE RESULTS
                new_trade_results_df = new_trade_results(scrip_df, twentieth_percentile_long, twentieth_percentile_short)

                # Separate new trade results into long and short
                new_long_trade_results_df = new_trade_results_df[new_trade_results_df['Type of Signal'] == 'Long'].reset_index(drop=True)
                new_short_trade_results_df = new_trade_results_df[new_trade_results_df['Type of Signal'] == 'Short'].reset_index(drop=True)
                # Separate new long and short trade results into two dataframes based on 20th Percentile achieved or not
                new_long_20th_achieved_df = new_long_trade_results_df[new_long_trade_results_df['20th Percentile Reached?'] == 'YES'].reset_index(drop=True)
                new_long_20th_not_achieved_df = new_long_trade_results_df[new_long_trade_results_df['20th Percentile Reached?'] == 'NO'].reset_index(drop=True)
                new_short_20th_achieved_df = new_short_trade_results_df[new_short_trade_results_df['20th Percentile Reached?'] == 'YES'].reset_index(drop=True)
                new_short_20th_not_achieved_df = new_short_trade_results_df[new_short_trade_results_df['20th Percentile Reached?'] == 'NO'].reset_index(drop=True)


                #GENERATE SUCCESS AND FAILURE SHEETS
                long_success, short_success = success_sheet(scrip_df, new_long_20th_achieved_df, new_short_20th_achieved_df, twentieth_percentile_long, twentieth_percentile_short)
                long_failure, short_failure = failure_sheet(scrip_df, new_long_20th_not_achieved_df, new_short_20th_not_achieved_df, twentieth_percentile_long, twentieth_percentile_short)

                #Generate final output sheets for long and short trades

                # Initialize new DataFrames for final output, copying the analysis dataframes
                final_output_long_df = long_analysis_df.copy()
                final_output_short_df = short_analysis_df.copy()

                # Calculate the metrics for final output
                time_highest_point_80th_long, time_20th_percentile_80th_long, lowest_point_20th_achieved_20th_long, lowest_point_20th_not_achieved_20th_long = final_output_long(new_long_trade_results_df, long_success, long_failure)
                time_lowest_point_80th_short, time_20th_percentile_80th_short, highest_point_20th_achieved_20th_short, lowest_point_20th_not_achieved_20th_short = final_output_short(new_short_trade_results_df, short_success, short_failure)

                # Add the new metrics to the analysis dataframes
                final_output_long_df['80th Percentile Time of Highest Point'] = time_highest_point_80th_long
                final_output_long_df['80th Percentile Time of 20th Percentile'] = time_20th_percentile_80th_long
                final_output_long_df['20th Percentile of Lowest Point (20th Achieved)'] = lowest_point_20th_achieved_20th_long
                final_output_long_df['20th Percentile of Lowest Point (20th Not Achieved)'] = lowest_point_20th_not_achieved_20th_long

                final_output_short_df['80th Percentile Time of Lowest Point'] = time_lowest_point_80th_short
                final_output_short_df['80th Percentile Time of 20th Percentile'] = time_20th_percentile_80th_short
                final_output_short_df['20th Percentile of Highest Point (20th Achieved)'] = highest_point_20th_achieved_20th_short #Changed Lowest to Highest
                final_output_short_df['20th Percentile Lowest Point (20th Not Achieved)'] = lowest_point_20th_not_achieved_20th_short

                # Store the analysis results for all stocks
                all_long_final_output.append(final_output_long_df)
                all_short_final_output.append(final_output_short_df)

    except Exception as e:
        print(f"Error processing {stock_symbol}: {e}")
        continue

1. Processing SAIL...
2. Processing SBICARD...
3. Processing SBILIFE...
4. Processing SBIN...
5. Processing SHREECEM...
6. Processing SHRIRAMFIN...
7. Processing SIEMENS...
8. Processing SRF...
9. Processing SUNPHARMA...
10. Processing SUNTV...
11. Processing SYNGENE...
12. Processing TATACHEM...
13. Processing TATACOMM...
14. Processing TATACONSUM...
15. Processing TATAMOTORS...
16. Processing TATAPOWER...
17. Processing TATASTEEL...
18. Processing TCS...
19. Processing TECHM...
20. Processing TITAN...
21. Processing TORNTPHARM...
22. Processing TRENT...
23. Processing TVSMOTOR...
24. Processing UBL...
25. Processing ULTRACEMCO...
26. Processing UPL...
27. Processing VEDL...
28. Processing VOLTAS...
29. Processing WIPRO...
30. Processing ZEEL...


In [ ]:
# Concatenate the analysis results for all stocks into two DataFrames
final_long_output_df = pd.concat(all_long_final_output, ignore_index=True)
final_short_output_df = pd.concat(all_short_final_output, ignore_index=True)

In [ ]:
# Save analysis to Excel
output_file_path = '/content/drive/MyDrive/Analysis/Categorize Trades/AllStocks_MACD_Analysis (Part 2).xlsx'

In [ ]:
with pd.ExcelWriter(output_file_path, engine='openpyxl', mode='w') as writer:
    final_long_output_df.to_excel(writer, sheet_name='Final Output Long', index=False)
    final_short_output_df.to_excel(writer, sheet_name='Final Output Short', index=False)

In [ ]:
# Confirm file save
print(f"Data saved to {output_file_path}")